# **What genres do the best in theatres?**
- Motivation:
    - Creating movies in popular genres will attract more initial viewers. It’s what the people are interested in watching.


**Smaller questions:**
- Which Ratings corrolate to those movies that generate the most money?
- Which genre is the least lucrative? And the most?
    - Which movie had the best investment return ratio? And which Genres normally do well?
    - Which movies generated the most money in their country of origin?

bus unde
data analysis
conclussion
data under
data prep
modeli
evaluation
deploy

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

Assign all the CSV files to something and add comments to it

In [5]:
#Titles, Studio, Domestic Gross, Foreign Gross and Year
#3387 rows
movies_gross = pd.read_csv("zippedData/bom.movie_gross.csv")

#Nconst, Primary Name, Primary Profession, Known for titles(title_id)
#606648 rows
basics_names = pd.read_csv("zippedData/name.basics.csv")

#Title ID's, Ordering, Title, Region, Language, Types, Attributes, Original Title
#331703 rows
akas = pd.read_csv("zippedData/title.akas.csv")

#Tconst, Primary Title, Original Title, Start Year, Runtime Minutes, Genres
#146144 rows
basics_titles = pd.read_csv("zippedData/title.basics.csv")

#Tconst, Directors, Writers
#146144 rows
crew = pd.read_csv("zippedData/title.crew.csv")

#Tconst, Ordering, Nconst, Category, Job, Characters
#1028186 rows
principals = pd.read_csv("zippedData/title.principals.csv")

#Tconst, Average Rating, Number of Votes
#73856 rows
ratings = pd.read_csv("zippedData/title.ratings.csv")

#Genre Id, Id, Original Language, Original Language, Original Title, Popularity, Release Date, Title, Vote Average, Vote count 
#26517 rows
movies = pd.read_csv("zippedData/tmdb.movies.csv")

#Id, Release Date, Movie Name, Production Budget, Domestic Gross, Worldwide Gross
#5782 rows
movies_budgets = pd.read_csv("zippedData/tn.movie_budgets.csv")

# tconst = Movie
# nconst = Person

Assign all the TSV files to something and add comments to it

In [6]:
#Id, Synopsis, Rating, Genre, Director, Writer, Theater Date, Dvd Date, Currency, Box Office, Runtime, Studio
#1560 rows 
movie_info = pd.read_csv("zippedData/rt.movie_info.tsv", sep='\t')

#Id, Reviews, Rating, Fresh, Critic, Top Critic, Publisher, Date
#54432 rows
reviews = pd.read_csv("zippedData/rt.reviews.tsv", sep='\t', encoding = 'unicode_escape')


2009 from 2019, treat 2020 as a special year


# Working with the basics_titles dataframe

Creating a table that contains Titles, Genres, Start Year and Tconst (cleaning some columns)

In [7]:
# Cleaning basics_titles to only those released after 2008
# Cleaning again to only those released before 2020
# Cleaning NAN values
# Using split to create a new column with separeted Genres
# Droping the unused Genres columns

a = basics_titles[basics_titles['start_year'] > 2008]
b = a[a['start_year'] <= 2019]
title_genre = b.drop(['original_title', 'runtime_minutes'], axis=1)
title_genre.dropna(inplace=True)

In [8]:
#title_genre.sort_values(['start_year'], ascending=False)
title_genre.head()

,tconst,primary_title,start_year,genres
0,tt0063540,Sunghursh,2013,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,Drama
3,tt0069204,Sabse Bada Sukh,2018,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy"


Creating a new table with the rating, tconst, primary_title and genre

In [9]:
# Joining tables to create a new one with more info
# Dropping NAN values
titlegenrerating = ratings.join(title_genre.set_index('tconst'), on='tconst')
titlegenrerating.dropna(inplace=True)

In [10]:
titlegenrerating.head(2)

,tconst,averagerating,numvotes,primary_title,start_year,genres
0,tt10356526,8.3,31,Laiye Je Yaarian,2019.0,Romance
1,tt10384606,8.9,559,Borderless,2019.0,Documentary


# Working with the movies_budgets dataframe

The goal here is to create a table with Movies Name, Budget, Domestic Gross and Worldwide Gross(Cleaning the column ID)

In [11]:
# Cleaning NAN values
# Renaming Column to match the other table, to make left-join possible

movies_money = movies_budgets.drop(['id', 'release_date'], axis=1)
movies_money.dropna(inplace=True)
movies_money.rename(columns={"movie": "primary_title"}, inplace=True)

In [12]:
#5782 rows
movies_money.head(2)

,primary_title,production_budget,domestic_gross,worldwide_gross
0,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [13]:
# Using method str.replace to remove ',' and '$' from the data columns 
# Using astype to change the values in those columns from String to Float because to Int gives an error

movies_money['worldwide_gross'] = movies_money['worldwide_gross'].str.replace(',', '')
movies_money['worldwide_gross'] = movies_money['worldwide_gross'].str.replace('$', '')
movies_money['worldwide_gross'] = movies_money['worldwide_gross'].astype(float)

movies_money['production_budget'] = movies_money['production_budget'].str.replace(',', '')
movies_money['production_budget'] = movies_money['production_budget'].str.replace('$', '')
movies_money['production_budget'] = movies_money['production_budget'].astype(float)

movies_money['domestic_gross'] = movies_money['domestic_gross'].str.replace(',', '')
movies_money['domestic_gross'] = movies_money['domestic_gross'].str.replace('$', '')
movies_money['domestic_gross'] = movies_money['domestic_gross'].astype(float)


In [14]:
# Creating a new column to give the investment return (How many dollars you get per dollar invested)
# Checking for duplicated entries on the data based on the primary title column q
# Dropping all the duplicates, not keeping any so it doesn't conflict with the other table and we get false information
# Checking one last time to see if we still have duplicates

movies_money['investment_return'] = movies_money['worldwide_gross']/movies_money['production_budget']
movies_money['investment_return'] = movies_money['investment_return'] - 1

duplicates_money = movies_money[movies_money.duplicated(subset=['primary_title'],keep=False)].sort_values(['primary_title'])
duplicates_money.head(6)

movies_money.drop_duplicates(subset='primary_title', keep=False, inplace=True)


In [15]:
movies_money.head(2)

,primary_title,production_budget,domestic_gross,worldwide_gross,investment_return
0,Avatar,425000000.0,760507625.0,2.776345e+09,5.532577
1,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,1.546673


# Creating the new table moviesfullinfo

The idea here is to join two tables to make one with ratings and money to make mostly a correlation between these 2 metrics

In [16]:
# Dropping Tconst, since this one is supposed to be the final table to use for graphs
# Dropping nan's values
# Creating a filter to only movies that had a budget over a million dollars

#new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])

moviesfullinfo = movies_money.join(titlegenrerating.set_index('primary_title'), on='primary_title')
moviesfullinfo.drop(['tconst'], axis=1, inplace=True)
moviesfullinfo = moviesfullinfo.dropna()
moviesfullinfo = moviesfullinfo[moviesfullinfo['production_budget'] >= 1000000]
moviesfullinfo = moviesfullinfo[moviesfullinfo['worldwide_gross'] >= 1]

In [17]:
#2410 rows
moviesfullinfo.head(3)

,primary_title,production_budget,domestic_gross,worldwide_gross,investment_return,averagerating,numvotes,start_year,genres
0,Avatar,425000000.0,760507625.0,2.776345e+09,5.532577,6.1,43.0,2011.0,Horror
1,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,1.546673,6.6,447624.0,2011.0,"Action,Adventure,Fantasy"
2,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-0.572108,6.0,24451.0,2019.0,"Action,Adventure,Sci-Fi"


# Creating the new table movieswoutratings

Here the goal was to make a table to make a correlation between Genre, Production, Worldwide Gross, Domestic Gross and ROI's

In [18]:
# Using method str.replace to remove ',' and '$' from the data columns 
# Using astype to change the values in those columns from String to Float because to Int gives an error

movieswoutratings = moviesfullinfo
movieswoutratings = movieswoutratings.dropna()

movieswoutratings = movieswoutratings[movieswoutratings['production_budget'] >= 1000000]

movieswoutratings = movieswoutratings[movieswoutratings['production_budget'] >= 1000000]
movieswoutratings = movieswoutratings[movieswoutratings['worldwide_gross'] >= 1]

In [19]:
#3392 rows
movieswoutratings.head()

,primary_title,production_budget,domestic_gross,worldwide_gross,investment_return,averagerating,numvotes,start_year,genres
0,Avatar,425000000.0,760507625.0,2.776345e+09,5.532577,6.1,43.0,2011.0,Horror
1,Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09,1.546673,6.6,447624.0,2011.0,"Action,Adventure,Fantasy"
2,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-0.572108,6.0,24451.0,2019.0,"Action,Adventure,Sci-Fi"
3,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,3.243841,7.3,665594.0,2015.0,"Action,Adventure,Sci-Fi"
6,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,5.827114,8.5,670926.0,2018.0,"Action,Adventure,Sci-Fi"


# Making tables for each genre

Created a table for each genre, so it's easier to see and manipulate

In [255]:
listatest = []
generos = "action, adventure, animation, biography, crime, documentary, drama, family, fantasy, history, horror, music, musical, mystery, romance, sci-fi, sport, thriller, war, western"
listatest.append(generos.split(","))
listatest = listatest[0]

In [256]:
listatest[15]

' sci-fi'

In [257]:
def get_df_genre(genre):
    '''
    This function gets a dataset based on the Genre
    '''
    return moviesfullinfo[moviesfullinfo['genres'].str.contains(genre.title().strip())]

In [258]:
all_genre_df = []
for genre in listatest:
    all_genre_df.append(get_df_genre(genre))
    

In [260]:
all_genre_df[15].head()

,primary_title,production_budget,domestic_gross,worldwide_gross,investment_return,averagerating,numvotes,start_year,genres
2,Dark Phoenix,350000000.0,42762350.0,1.497624e+08,-0.572108,6.0,24451.0,2019.0,"Action,Adventure,Sci-Fi"
3,Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09,3.243841,7.3,665594.0,2015.0,"Action,Adventure,Sci-Fi"
6,Avengers: Infinity War,300000000.0,678815482.0,2.048134e+09,5.827114,8.5,670926.0,2018.0,"Action,Adventure,Sci-Fi"
13,John Carter,275000000.0,73058679.0,2.827781e+08,0.028284,6.6,241792.0,2012.0,"Action,Adventure,Sci-Fi"
16,Captain America: Civil War,250000000.0,408084349.0,1.140069e+09,3.560278,7.8,583507.0,2016.0,"Action,Adventure,Sci-Fi"


In [261]:
def avg_rating_genre(dataframe):
    '''
    This function gets the avarage(mean) rating based on the Genre
    '''
    return(dataframe['averagerating'].mean())


In [262]:
avg_rating_per_genre1 = []
for rating in all_genre_df:
    avg_rating_per_genre1.append(avg_rating_genre(rating))

In [263]:
avg_rating_per_genre1

[6.258284600389862,
 6.487146529562983,
 6.556410256410256,
 6.998265895953759,
 6.370805369127515,
 7.112142857142863,
 6.503302286198149,
 6.130088495575223,
 6.064925373134325,
 6.783333333333336,
 5.589999999999997,
 6.423287671232873,
 6.306666666666668,
 6.112426035502957,
 6.242962962962962,
 6.351874999999999,
 6.549999999999998,
 6.021818181818183,
 6.528571428571428,
 6.7153846153846155]

In [264]:
def avg_roi_genre(dataframe):
    '''
    This function gets the ROI based on the Genre
    '''
    worldgross = sum(dataframe['worldwide_gross'])
    prodbudget = sum(dataframe['production_budget'])
    print(prodbudget)
    return (worldgross/prodbudget-1)*100


In [265]:
avg_roi_per_genre = []
for genre_df in all_genre_df:
    avg_roi_per_genre.append(avg_roi_genre(genre_df))

36530200000.0
37419383106.0
10732500000.0
4717000479.0
9230207373.0
4138343106.0
30528537142.0
6430700000.0
9999500000.0
2198400000.0
5772267373.0
1454450000.0
473200000.0
4256855023.0
5670990000.0
13167157373.0
1304400000.0
12389042673.0
849500000.0
665800000.0


In [173]:
avg_roi_genre(documentary)

159.6447131563673


In [172]:
str(listatest)

"['action', ' adventure', ' animation', ' biography', ' crime', ' documentary', ' drama', ' family', ' fantasy', ' history', ' horror', ' music', ' musical', ' mystery', ' romance', ' scifi', ' sport', ' thriller', ' war', ' western']"

In [76]:
avg_roi_per_genre = [198.8863695408183, 230.17935080599776, 269.3128482366643, 187.1707223966979, 150.52689770158648, 159.6447131563673 ]

    
    

'action'

In [ ]:
action = moviesfullinfo[moviesfullinfo['genres'].str.contains("Action")]
adventure = moviesfullinfo[moviesfullinfo['genres'].str.contains("Adventure")]
animation = moviesfullinfo[moviesfullinfo['genres'].str.contains("Animation")]
biography = moviesfullinfo[moviesfullinfo['genres'].str.contains("Biography")]
crime = moviesfullinfo[moviesfullinfo['genres'].str.contains("Crime")]
documentary = moviesfullinfo[moviesfullinfo['genres'].str.contains("Documentary")]
drama = moviesfullinfo[moviesfullinfo['genres'].str.contains("Drama")]
family = moviesfullinfo[moviesfullinfo['genres'].str.contains("Family")]
fantasy = moviesfullinfo[moviesfullinfo['genres'].str.contains("Fantasy")]
history = moviesfullinfo[moviesfullinfo['genres'].str.contains("History")]
horror = moviesfullinfo[moviesfullinfo['genres'].str.contains("Horror")]
music = moviesfullinfo[moviesfullinfo['genres'].str.contains("Music")]
musical = moviesfullinfo[moviesfullinfo['genres'].str.contains("Musical")]
mystery = moviesfullinfo[moviesfullinfo['genres'].str.contains("Mystery")]
romance = moviesfullinfo[moviesfullinfo['genres'].str.contains("Romance")]
scifi = moviesfullinfo[moviesfullinfo['genres'].str.contains("Sci-Fi")]
sport = moviesfullinfo[moviesfullinfo['genres'].str.contains("Sport")]
thriller = moviesfullinfo[moviesfullinfo['genres'].str.contains("Thriller")]
war = moviesfullinfo[moviesfullinfo['genres'].str.contains("War")]
western = moviesfullinfo[moviesfullinfo['genres'].str.contains("Western")]


# To Do List TODAY

Try to go with scatter plot with legends OR Bar Chart with ROI's?

Make that table with the years only, so we have determine the period 2009 - 2019

Join tables by both const and year

# Visualization
### Rating per genre
- How likely is to have a better rating with the critics if your movie is a certain genre?
    - Is there a best combination of Genre and Investment to get the best rating?


> To answer that we use "moviesfullinfo" table that we prepared for this topic!

In [1]:
# Tirar o avarage rating per genre
# Plotar grafico

# Fazer uma correlacao entre investimento e rating (x Money / 1 whole point?)
# Is there a "floor", that after you break it,
# you can presume that you get at least a 6?

In [ ]:
sns.set_theme(style="ticks")

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(7, 10))
ax.set_xscale("log")

# Load the example planets dataset
genre = sns.load_dataset("planets")

# Plot the orbital period with horizontal boxes
sns.boxplot(x="distance", y="method", data=planets,
            whis=[0, 100], width=.6, palette="vlag")

# Add in points to show each observation
sns.stripplot(x="Avg. Rating per Genre", y="Genre", data=planets,
              size=4, color=".3", linewidth=0)

# Tweak the visual presentation
ax.xaxis.grid(True)
ax.set(ylabel="")
sns.despine(trim=True, left=True)

### Money per genre
- What is the best genre solely focused on how much money you can make?
- What genre is the "safest choice"? Less likely to lose money?
- What genre takes the least money to produce? And the most?


```python


intakes['event_num'] = intakes.sort_values(['intake_date']).groupby(['animal_id']).rank()
outcomes['event_num'] = outcomes.sort_values(['outcome_date']).groupby(['animal_id']).rank()
```

### Minutes per genre
- Are there genres that tend to last longer?
- Lasting longer gives the movies more chance to profit more? How about ratings?